In [1]:
from utils import PredictorSPlitModel, arr2db, forna_url, DataEncoder

Using TensorFlow backend.


In [2]:
# running on CPU is extremely slow!

In [3]:
import numpy as np

In [9]:
from collections import Counter

In [4]:
import keras

In [5]:
# model = Predictor('model/2019_09_08.1/model.hdf5')
model = PredictorSPlitModel('model/2019_09_12.1/model.hdf5')

W0930 11:23:01.631141 4577457600 deprecation_wrapper.py:119] From /Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0930 11:23:01.718617 4577457600 deprecation_wrapper.py:119] From /Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0930 11:23:01.720088 4577457600 deprecation_wrapper.py:119] From /Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0930 11:23:01.799027 4577457600 deprecation_wrapper.py:119] From /Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:

4


W0930 11:23:04.407119 4577457600 deprecation_wrapper.py:119] From /Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0930 11:23:04.853574 4577457600 deprecation.py:323] From /Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/tensorflow/python/ops/math_grad.py:1250: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0930 11:23:10.682780 4577457600 utils.py:343] Model missing names layers, will try to infer (unreliable!).
utils.py:365: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
  output=[layer_final_hidden.output, layer_fe.output])
utils.py:375: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor

In [6]:
seq='GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU'
y, logp, fe = model.predict_one_step_ar(seq=seq, 
                                       n_sample=100, start_offset=2, p_clip=1e-12)



 12%|█▎        | 4/32 [00:05<00:41,  1.48s/it]utils.py:482: RuntimeWarning: divide by zero encountered in log
  _lp = vals_sampled * np.log(_vals) + (1 - vals_sampled) * np.log(1 - _vals)
100%|██████████| 32/32 [00:41<00:00,  1.26s/it]


In [7]:
y.shape

(100, 34, 34, 1)

In [10]:
db_strs = []  # collect all dot bracket notation

for idx_sample in range(y.shape[0]):  
    _pred = y[idx_sample, :, :, 0]

#     print idx_sample
#     print np.where(_pred==1)
    
#     # locate pseudo knot?
#     one_idxes = zip(np.where(_pred==1)[0], np.where(_pred==1)[1])
#     for i in range(len(one_idxes)):
#         for j in range(i+1, len(one_idxes)):
#             pair1 = one_idxes[i]
#             pair2 = one_idxes[j]
#             # see if they cross
#             if pair2[0] < pair1[1] < pair2[1]:
#                 print 'knot!'
#                 print pair1, pair2
    

    
    # dot-bracket notation
    db_str = arr2db(_pred)
    db_strs.append(db_str)
#     print idx_sample, db_str
    print idx_sample
    print forna_url(seq, db_str)
    
    print ''

[[(0, 18), (1, 17), (2, 16), (3, 15), (4, 14), (5, 12), (6, 11)]]
[0]
0
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU&structure=(((((((....)).)))))...............

[[(0, 18), (1, 17), (2, 16), (3, 15), (4, 14), (5, 12), (6, 11)]]
[0]
1
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU&structure=(((((((....)).)))))...............

[[(0, 18), (1, 17), (2, 16), (3, 15), (4, 14), (5, 12), (6, 11)]]
[0]
2
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU&structure=(((((((....)).)))))...............

[[(0, 18), (1, 17), (2, 16), (3, 15), (4, 14), (5, 12), (6, 11)]]
[0]
3
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU&structure=(((((((....)).)))))...............

[[(0, 18), (1, 17), (2, 16), (3, 15), (4, 14), (5, 12), (6, 11)]]
[0]
4
http://nibiru.tbi.univie.ac.at/forna/for

In [12]:
db_str_ct = Counter(db_strs)
print db_str_ct

Counter({'(((((((....)).)))))...............': 61, '...((((....)).)).((........)).....': 24, '..................................': 4, '(((((.........)))))...............': 4, '................(((........)))....': 3, '......(((((((..............)))))))': 2, '...((.........)).((........)).....': 1, '[[[[[..((((((.]]]]]........)))))).': 1})


In [14]:
for _s, _c in db_str_ct.iteritems():
    print _c
    print forna_url(seq, _s)
    print ''

24
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU&structure=...((((....)).)).((........)).....

61
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU&structure=(((((((....)).)))))...............

4
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU&structure=..................................

4
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU&structure=(((((.........)))))...............

2
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU&structure=......(((((((..............)))))))

1
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU&structure=...((.........)).((........)).....

3
http://nibiru.tbi.univie.ac.at/forna/forna.html?id=url/name&sequence=GGCGCAGUGGGCUAG